In [1]:
# This program opens a certain number of files. Given a period of time, the program will attempt
# to split the file in equal intervals of duration equal to that period.
# Then, a fft is performed for each interval. Then, all of these are averaged and plotted.
#Finally, the program performs the fft of the whole file without splitting and plots the spectrograph aswell.

In [2]:
# Libraries and packages needed

%matplotlib widget

import winsound
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
import pandas as pd
import math
from __future__ import division
from numpy import linspace, loadtxt, ones, convolve
import numpy as numpy
import logging

In [3]:
#Logging configuration. I recommend setting DEBUG to INFO for less debugging info :)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# Constants
CFREQ = 523
DFREQ = 261
DUR = 200

In [5]:
#Functions

def Beep():
    winsound.Beep(CFREQ,DUR)
    winsound.Beep(DFREQ,DUR)
    
def IsNumber(string):                     # Check if a string is a number
    if not string:  # Handle empty strings
        return False
    try:
        # Convert the string to a float, this will validate both integers and floats
        float(string)
        return True
    except ValueError:
        # If conversion fails, it's not a number
        return False
        
def GetMaxValueMaxIndex(values,repeat):
    max_values = []
    max_indices = []
    for i in range(0, math.floor(len(values)/repeat),1):
        segment = values[math.floor(i * repeat) : math.floor((i+1) * repeat)]
        max_value = np.max(segment)
        max_index = np.argmax(segment) + math.floor(i * repeat)  # Add i to get the absolute index in the original array
        max_values.append(max_value)
        max_indices.append(max_index)
    max_values = np.array(max_values)
    max_indices = np.array(max_indices)
    return max_values, max_indices

def AskInput(expectedType):
    Beep()
    while True:
        value = input()
        if expectedType == int or expectedType == float:
            if not IsNumber(value):
                print("Wrong input, try again")
            else:
                #Borrar esta linea de abajo
                return value
                
                print(f"Your input is: {value}. Do you want to change it? (y=yes,n=no)")
                check = input()
                if check == 'n':
                    return value
        else:
            #Y esta
            return value
            
            print(f"Your input is: {value}. Do you want to change it? (y=yes,n=no)")
            check = input()
            if check == 'n':
                return value

def GetPath():
    path = AskInput(str)
    
    if os.path.isfile(path):
        logging.debug(f"The provided path is a valid file: {path}")
    elif os.path.isdir(path):
        logging.debug(f"The provided path is a valid directory: {path}")
        
        path = os.path.join(path, '')
    else:
        logging.error("The path provided is neither a valid file nor a directory.")
    
    return path

def ExtractIntensities(frequencies, intensities, freqStart, freqEnd):
    # Convert input lists to numpy arrays if they aren't already
    frequencies = np.array(frequencies)
    intensities = np.array(intensities)
    
    # Find indices where frequencies are within the specified range
    indices = np.where((frequencies >= freqStart) & (frequencies <= freqEnd))[0]
    
    # Check if there are any frequencies in the range
    if len(indices) == 0:
        raise ValueError("No frequencies found within the specified range.")
    
    # Extract intensities for these indices
    selected_intensities = intensities[indices]
    return selected_intensities




def stdev_intensity(frequencies, intensities, freq_start, freq_end):
    # Convert input lists to numpy arrays if they aren't already
    frequencies = np.array(frequencies)
    intensities = np.array(intensities)
    
    # Find indices where frequencies are within the specified range
    indices = np.where((frequencies >= freq_start) & (frequencies <= freq_end))[0]
    
    # Check if there are any frequencies in the range
    if len(indices) == 0:
        raise ValueError("No frequencies found within the specified range.")
    
    # Extract intensities for these indices
    selected_intensities = intensities[indices]
    
    # Calculate and return the average intensity
    stand_deviat = np.std(selected_intensities)
    return stand_deviat



def SignalToNoise(fArray, iArray):
    #print(f"Frequency window of noise (beginning): ")
    #winBeg = float(AskInput(float))
    #print(f"Frequency window of noise (end): ")
    #winEnd = float(AskInput(float))

    winBeg = 22.0
    winEnd = 22.5
    
    stand_deviat = stdev_intensity(fArray, iArray, winBeg, winEnd)

    return 1 / stand_deviat


def MovingAverage(interval, window_size):
    window= numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

Beep()

In [6]:
#### Some general info

print("Number of folders: ")
numFolders = int(AskInput(int))
print("Common name of files: ")
commonName = AskInput(str)
print("Sampling rate (Msamp/s): ")
sampRate = float(AskInput(float)) * 10**6
print("Minimum frequency of interest (MHz): ")
minFreqIntPlot = float(AskInput(float))
print("Maximum frequency of interest (MHz): ")
maxFreqIntPlot = float(AskInput(float))

Beep()

Number of folders: 


 1


Common name of files: 


 B_


Sampling rate (Msamp/s): 


 250


Minimum frequency of interest (MHz): 


 0


Maximum frequency of interest (MHz): 


 250


In [7]:
## Where everything comes together
with open("SNRAnalysisWithAmp.txt", "w") as snrFile:
    
    for i in range(0, numFolders):
        ## Asks for folders. One folder per setup
        print(f"*****************ACCESSING FOLDER {i + 1}*****************")
        print(f"Please, introduce the path of the folder {i + 1}, where files are: ")
        pathFolder = GetPath()
        
        print(f"Number of files in this folder: ")
        numFiles = int(AskInput(int))
        
        # Info of each setup
        print(f"Repeating frequency of FIRST comb (Hz) for folder {i + 1}: ")
        rep1 = float(AskInput(float))
        
        print(f"Repeating frequency of SECOND comb (Hz) for folder {i + 1}: ")
        rep2 = float(AskInput(float))
        
        df = abs(rep2 - rep1)
        print("Off-set frequency (MHz): ")
        f_ceo = float(AskInput(float) * 10 ** 6)
        print("Approx example optical frequency (THz): ")
        f_opt_wanted = float(AskInput(float)) * 10 ** 12
        
        f_unamb = (rep1 ** 2) / (2 * df)  # unambiguity range
    
        for k in range(1, 10):
            if f_opt_wanted < k * f_unamb:
                unamb_n = k
                break
    
        logging.info(f"Difference of repetition rates for folder {i + 1}: {df} Hz")
        logging.info(f'The unambiguity range is {f_unamb / 10 ** 12} THz')
        logging.info(f"The order of the unambiguity region is {unamb_n}")
    
        fftsFull = None
    
        print(f"Plotting results of folder {i+1}. Setup name: ")
        setup = AskInput(str)

        
        ## Access each of the files inside the folder.
        for j in range(0, numFiles):
            filePath = pathFolder + commonName + str(j + 1) + ".h5"
            logging.debug(f"Opening and reading file {filePath}")
    
    
            # Copy the data inside the file
            data = h5py.File(filePath, "r")
            signal = np.array(data[commonName + str(j + 1)])
            logging.debug(signal)
            logging.debug(len(signal))
            data.close()
    
            
            logging.debug(f"Duration of the measurement: {len(signal) / sampRate} seconds.")
    
    
    
            ########################### FOR THE SPLIT SIGNAL AND ITS FFT #############################
            # Calculate the length of each subdivision so every subdivision has one pulse, approx
            lengthSubdivision = int(math.ceil(sampRate / df))
            logging.debug(f"Array length: {lengthSubdivision}")
    
            
            # Calculate amount of segments and trim the signal so each division will have the same length
            numFullSegments = len(signal) // lengthSubdivision
            trimmedSignal = signal[:numFullSegments * lengthSubdivision]
            logging.debug(signal)
            logging.debug(len(signal))
            logging.debug(trimmedSignal)
            logging.debug(
                f"Lost data points: {len(signal) - len(trimmedSignal)}, representing a {100 * (len(signal) - len(trimmedSignal)) / len(signal)} % of the total"
            )
    
            
            # Split the whole signal into arrays with the length calculated earlier
            subdivArray = trimmedSignal.reshape(-1, lengthSubdivision)
            logging.debug(subdivArray)
    
            
            # Perform FFT of each subdivision
            ffts = np.abs(fft(subdivArray))
            ffts = np.array(ffts)
            logging.debug(ffts)
    
            
            # Average all ffts
            avg = np.zeros(len(ffts[0]))
            for subdiv in ffts:
                avg += subdiv / len(ffts)
    
    
            logging.debug(avg)
            #############################################################################################
    
            
            ############################# FOR THE FFT OF THE WHOLE SIGNAL ################################
    
            # Perform FFT on the entire signal
            fft_signal = np.abs(fft(signal))
            fft_signal = np.array(fft_signal)
    
            if j == 0:
                fftsFull = fft_signal
            else:
                fftsFull += fft_signal
            
            # Arrays of frequencies
            freqArray = np.linspace(0, sampRate / (10 ** 6), len(avg))
            logging.debug(freqArray)
            fvec = np.linspace(0, sampRate / (10 ** 6), len(fft_signal))
            
            ##############################################################################################
    
            ######################### CUT FREQUENCIES WHICH ARE OUTSIDE OF THE SPECIFIED RANGE #################
    
            avgForPlot = ExtractIntensities(freqArray, avg, minFreqIntPlot, maxFreqIntPlot)
            fftSignalForPlot = ExtractIntensities(fvec, fft_signal, minFreqIntPlot, maxFreqIntPlot)
    
            freqArrayForPlot = np.linspace(minFreqIntPlot,maxFreqIntPlot, len(avgForPlot))
            fvecForPlot = np.linspace(minFreqIntPlot,maxFreqIntPlot, len(fftSignalForPlot))
            
            ############################## FOR PLOTTING STUFF ############################################
            
            avgForPlot = avgForPlot/max(avgForPlot)
            fftSignalForPlot = fftSignalForPlot/max(fftSignalForPlot)
    
    
    
            # Plot results
    
            plotName = f"Setup: {setup}, file B_{j+1}"
    
            # Create a new figure
            plt.figure()
    
            # Plot results
            plt.title(plotName)
            plt.plot(freqArrayForPlot, avgForPlot, label=f"Signal split in {1000/df} ms, then FFTd, then averaged.")
            plt.ylabel('Signal (arb. unit)', fontsize=20)
            plt.xlabel('Radio frequency (MHz)', fontsize=20)
            plt.xlim([minFreqIntPlot, maxFreqIntPlot])
            
            plt.plot(fvecForPlot, fftSignalForPlot, label=f"FFT of setup {setup}, file B_{j+1}.")
            plt.legend(loc='upper left')
            
            plt.show()
    
            logging.info(f"Setup: {setup}, File {j+1}")
            logging.debug(f"SNR when split in {1000/df} ms, then FFTd, then averaged: {SignalToNoise(freqArrayForPlot, avgForPlot)}.")
            logging.debug(f"SNR FFT of the whole signal: {SignalToNoise(fvecForPlot, fftSignalForPlot)}")
            
            # Write SNR results to the file instead of printing
            snrFile.write(f"{SignalToNoise(freqArrayForPlot, avgForPlot)}\t") # SNR when split in {1000/df} ms, then FFTd, then averaged
            snrFile.write(f"{SignalToNoise(fvecForPlot, fftSignalForPlot)}\n\n") #SNR FFT of the whole signal: 
            
        fftsFull = fftsFull/numFiles
        fftsFullForPlot = ExtractIntensities(np.linspace(0, sampRate / (10 ** 6), len(fftsFull)), fftsFull, minFreqIntPlot, maxFreqIntPlot)
        
        fFullVecForPlot = np.linspace(minFreqIntPlot,maxFreqIntPlot, len(fftsFullForPlot))
        fftsFullForPlot = fftsFullForPlot/max(fftsFullForPlot)
        
        plt.figure(int(k*j+1))
        plt.plot(fFullVecForPlot, fftsFullForPlot, label =f"Average of FFT of each file for setup {setup}")
        plt.legend(loc='upper left')
        plt.show()







        # Transfer to optical
        
        fvec = fFullVecForPlot
        ffts = fftsFullForPlot
        
        fr = fFullVecForPlot * 10 ** 6  #array of frequencies taken from the real data
        f_opt = [0 for i in range(len(fr))]  #new array for the optical wavelengths
            
            
        for l in range (len(fr)): #transferring each element of the fr (which is in Hz) to the optical frequency and writing into f_opt
            if (unamb_n % 2) == 0:
                n = (((rep1) / 2) - fr[l]) / df
            else:
                n = fr[l] / df
            nreg = (rep1 / 2) / df 
            n = n + (unamb_n - 1) * nreg 
            f_opt[l] = (n * rep1) / 10 ** 12

        
        f_opt = numpy.array(f_opt)
        logging.info(f_opt)

        f_opt = f_opt[f_opt != 0]
        logging.info(f_opt)
        #Get plots of intensities vs frequency in THz for all setups
        
        window = 1
            
        maxValueArray, maxIndexArray = GetMaxValueMaxIndex(ffts,df)
        
        avMax = MovingAverage(maxValueArray,window)
        
        avMax = avMax/np.max(avMax)

        
        plt.figure()
        plt.plot(f_opt[maxIndexArray], avMax, label = f"Average of FFTs of each file for setup {setup}. Optical range.")
        plt.ylabel('Signal (arb. unit)',fontsize=20)
        plt.xlabel('Frequency (THz)',fontsize=20)




        
        logging.debug(f"SNR FFT averaged for all files of setup {setup}: {SignalToNoise(fFullVecForPlot,fftsFullForPlot)}")
        snrFile.write(f"SNR FFT averaged for all files of setup {setup}: {SignalToNoise(fFullVecForPlot, fftsFullForPlot)}\n")
        snrFile.write("\n")
        
Beep()
Beep()
Beep()


*****************ACCESSING FOLDER 1*****************
Please, introduce the path of the folder 1, where files are: 


 C:\Users\Óscar\Desktop\DCS\2024_09_20_14h_17m_53s\Digitizer1


Number of files in this folder: 


 1000


Repeating frequency of FIRST comb (Hz) for folder 1: 


...

 422


2024-09-20 15:09:48,314 - INFO - Difference of repetition rates for folder 1: 321.50999999046326 Hz
2024-09-20 15:09:48,314 - INFO - The unambiguity range is 97.19782996942905 THz
2024-09-20 15:09:48,314 - INFO - The order of the unambiguity region is 5


Plotting results of folder 1. Setup name: 


 End Point


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
print(avgForPlot)

In [ ]:
print(GetTop20Values(avgForPlot))

In [ ]:
print(fft_signal)

In [ ]:
print(fftSignalForPlot)

In [ ]:
print(len(fftSignalForPlot))

In [ ]:
print(avgForPlot)

In [ ]:
len(avgForPlot)

In [ ]:
plt.plot(fvec,fft_signal)
plt.show()

In [ ]:
print(fft_signal)
print(len(fft_signal))

In [ ]:
int(math.ceil(sampRate / df))

In [ ]:
1000000*1000/sampRate

In [9]:
snrFile.write(f"SNR average of FFT in optical range for {setup}: {SignalToNoise(f_opt[maxIndexArray], avMax)}\n")

ValueError: No frequencies found within the specified range.

In [2]:
import random
import math

# Initialize the list
random_list = []

# Generate 100 rows
for _ in range(100):
    # First column: random integer between 1 and 10
    col1 = random.randint(1, 10)
    # Second column: random float between 0 and pi
    col2 = random.uniform(0, math.pi)
    # Third column: random float between 0 and 2*pi
    col3 = random.uniform(0, 2 * math.pi)
    
    # Append the row to the list
    random_list.append([col1, col2, col3])

# Print the generated list
for row in random_list:
    print(row)

[1, 1.4167225114924706, 0.42247601856863476]
[9, 3.1090139074191545, 1.3245534264675236]
[10, 3.0845819440098152, 5.376153746643218]
[10, 2.3962811847690224, 2.3708439124579446]
[10, 2.8834305622320904, 3.4144973607648397]
[8, 0.2501115278551743, 4.462578668724529]
[1, 0.7502684328092647, 1.0874531742183378]
[4, 0.6062135133549432, 1.4553684551011221]
[1, 3.077520331853799, 5.536219103121288]
[2, 0.5387737037589406, 2.033951466571307]
[8, 2.92851522384481, 4.174903126313331]
[6, 1.6554617126302136, 0.9821880426205356]
[7, 3.115453777493776, 3.4948812204956274]
[1, 1.0575775820363407, 2.2143768885929345]
[10, 1.6991089892533113, 3.887657759215811]
[1, 1.5143073039964254, 2.1710122300269985]
[10, 0.4274401517921132, 4.397718851477043]
[4, 2.493711537472407, 3.1445488366765546]
[5, 0.15935707429170204, 5.0920247691263825]
[4, 2.217937670593012, 2.087493610267329]
[1, 0.38452723308488274, 1.24294374405864]
[9, 2.473145097170866, 2.003053427575539]
[9, 1.832818210424529, 3.20312083131545]
[